In [ ]:
import csv

with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        print(row)

['AA', '39.48', '6/11/2007', '9:36am', '-0.18', '181800']
['AIG', '71.38', '6/11/2007', '9:36am', '-0.15', '195500']
['AXP', '62.58', '6/11/2007', '9:36am', '-0.46', '935000']
['BA', '98.31', '6/11/2007', '9:36am', '+0.12', '104800']
['C', '53.08', '6/11/2007', '9:36am', '-0.25', '360900']
['CAT', '78.29', '6/11/2007', '9:36am', '-0.23', '225400']


In [3]:
from collections import namedtuple

with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headings = next(f_csv)
    Row = namedtuple('Row', headings)
    for r in f_csv:
        row = Row(*r)
        print(row)

Row(Symbol='AA', Price='39.48', Date='6/11/2007', Time='9:36am', Change='-0.18', Volume='181800')
Row(Symbol='AIG', Price='71.38', Date='6/11/2007', Time='9:36am', Change='-0.15', Volume='195500')
Row(Symbol='AXP', Price='62.58', Date='6/11/2007', Time='9:36am', Change='-0.46', Volume='935000')
Row(Symbol='BA', Price='98.31', Date='6/11/2007', Time='9:36am', Change='+0.12', Volume='104800')
Row(Symbol='C', Price='53.08', Date='6/11/2007', Time='9:36am', Change='-0.25', Volume='360900')
Row(Symbol='CAT', Price='78.29', Date='6/11/2007', Time='9:36am', Change='-0.23', Volume='225400')


In [4]:
with open('stocks.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        print(row)

{'Symbol': 'AA', 'Price': '39.48', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.18', 'Volume': '181800'}
{'Symbol': 'AIG', 'Price': '71.38', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.15', 'Volume': '195500'}
{'Symbol': 'AXP', 'Price': '62.58', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.46', 'Volume': '935000'}
{'Symbol': 'BA', 'Price': '98.31', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '+0.12', 'Volume': '104800'}
{'Symbol': 'C', 'Price': '53.08', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.25', 'Volume': '360900'}
{'Symbol': 'CAT', 'Price': '78.29', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.23', 'Volume': '225400'}


In [5]:
headers = ['Symbol','Price','Date','Time','Change','Volume']
rows = [('AA', 39.48, '6/11/2007', '9:36am', -0.18, 181800),
        ('AIG', 71.38, '6/11/2007', '9:36am', -0.15, 195500),
        ('AXP', 62.58, '6/11/2007', '9:36am', -0.46, 935000)]

with open('stocks2.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

In [6]:
headers = ['Symbol', 'Price', 'Date', 'Time', 'Change', 'Volume']
rows = [{'Symbol':'AA', 'Price':39.48, 'Date':'6/11/2007', 'Time':'9:36am', 'Change':-0.18, 'Volume':181800},
        {'Symbol':'AIG', 'Price': 71.38, 'Date':'6/11/2007', 'Time':'9:36am', 'Change':-0.15, 'Volume': 195500},
        {'Symbol':'AXP', 'Price': 62.58, 'Date':'6/11/2007', 'Time':'9:36am', 'Change':-0.46, 'Volume': 935000}]

with open('stocks3.csv','w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

In [26]:
col_types = [str, float, str, str, float, int]

with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        # Применение преобразований к элементам строки
        row = tuple(convert(value) for convert, value in zip(col_types, row))
        print(row)

('AA', 39.48, '6/11/2007', '9:36am', -0.18, 181800)
('AIG', 71.38, '6/11/2007', '9:36am', -0.15, 195500)
('AXP', 62.58, '6/11/2007', '9:36am', -0.46, 935000)
('BA', 98.31, '6/11/2007', '9:36am', 0.12, 104800)
('C', 53.08, '6/11/2007', '9:36am', -0.25, 360900)
('CAT', 78.29, '6/11/2007', '9:36am', -0.23, 225400)


In [29]:
print('Reading as dicts with type conversion')
field_types = [('Price', float), ('Change', float), ('Volume', int)]

with open('stocks.csv') as f:
    for row in csv.DictReader(f):
        row.update((key, conversion(row[key])) for key, conversion in field_types)
        print(row)

Reading as dicts with type conversion
{'Symbol': 'AA', 'Price': 39.48, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.18, 'Volume': 181800}
{'Symbol': 'AIG', 'Price': 71.38, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.15, 'Volume': 195500}
{'Symbol': 'AXP', 'Price': 62.58, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.46, 'Volume': 935000}
{'Symbol': 'BA', 'Price': 98.31, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': 0.12, 'Volume': 104800}
{'Symbol': 'C', 'Price': 53.08, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.25, 'Volume': 360900}
{'Symbol': 'CAT', 'Price': 78.29, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.23, 'Volume': 225400}


In [43]:
from collections import OrderedDict
import json

s = '{"name": "ACME", "shares": 50, "price": 490.1}'
data = json.loads(s, object_pairs_hook=OrderedDict)
print(data)

class JSONObject:
    def __init__(self, d):
        self.__dict__ = d
        
data1 = json.loads(s, object_hook=JSONObject)
print(data1.name)
print(data1.shares)
print(data1.price)

OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
ACME
50
490.1


In [61]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
def serialize_instance(obj):
    d = {'__classname__' : type(obj).__name__}
    d.update(vars(obj))
    return d

# Словарь отображения имен на известные классы
classes = {'Point': Point}

def unserialize_object(d):
    clsname = d.pop('__classname__', None)
    if clsname:
        cls = classes[clsname]
        obj = cls.__new__(cls) # Создание экземпляра без вызова __init__
        for key, value in d.items():
            setattr(obj, key, value)
            return obj
    else:
        return d
    
p = Point(2, 3)
s = json.dumps(p, default=serialize_instance)
print(s)

a = json.loads(s, object_hook=unserialize_object)
print(a)

print(a.x)




{"__classname__": "Point", "x": 2, "y": 3}
2


In [62]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse

# Скачивание и парсинг RSS-канала
u = urlopen('http://planet.python.org/rss20.xml')
doc = parse(u)

# Извлечение и вывод нужных тегов
for item in doc.iterfind('channel/item'):
    title = item.findtext('title')
    date = item.findtext('pubDate')
    link = item.findtext('link')
    
    print(title)
    print(date)
    print(link)
    print()

Real Python: A Close Look at a FastAPI Example Application
Mon, 03 Nov 2025 14:00:00 +0000
https://realpython.com/fastapi-python-web-apis/

Reuven Lerner: Want to learn uv?
Mon, 03 Nov 2025 13:15:29 +0000
https://lerner.co.il/2025/11/03/want-to-learn-uv/

Real Python: Quiz: A Close Look at a FastAPI Example Application
Mon, 03 Nov 2025 12:00:00 +0000
https://realpython.com/quizzes/fastapi-python-web-apis/

eGenix.com: PyDDF Python Herbst Sprint 2025
Mon, 03 Nov 2025 09:00:00 +0000
https://www.egenix.com/company/news/PyDDF-Herbst-Sprint-2025

Hugo van Kemenade: Python Core Sprint 2025
Mon, 03 Nov 2025 08:58:00 +0000
https://hugovk.dev/blog/2025/python-core-sprint/

Python Bytes: #456 You're so wrong
Mon, 03 Nov 2025 08:00:00 +0000
https://pythonbytes.fm/episodes/show/456/youre-so-wrong

Django Weblog: Announcing DjangoCon Europe 2026 in Athens, Greece! 🏛️🇬🇷
Mon, 03 Nov 2025 07:00:00 +0000
https://www.djangoproject.com/weblog/2025/nov/03/announcing-djangocon-europe-2026-in-athens-greece/

In [70]:
print(doc)
print(doc.find('channel/title'))
print(doc.find('channel/title').tag)
print(doc.find('channel/title').text)

<Element 'title' at 0x0000020F59016390>
title
Planet Python


In [75]:
from xml.etree.ElementTree import Element, tostring

def dict_to_xml(tag, d):
    '''Превращает простой словарь пар ключ/значение в XML'''
    elem = Element(tag)
    for key, val in d.items():
        child = Element(key)
        child.text = str(val)
        elem.append(child)
    return elem

s = {'name': 'GOOG', 'shares': 100, 'price':490.1}
e = dict_to_xml('stock', s)

print(e)
print(tostring(e))

e.set('_id','1234')
print(tostring(e))

def dict_to_xml_str(tag, d):
    '''Превращает простой словарь пар ключ/значение в XML'''
    parts = [f'<{tag}>']
    for key, val in d.items():
        parts.append(f'<{key}>{val}</{key}>')
        parts.append(f'</{tag}>')
    return ''.join(parts)


d = {'name': '<spam>'}

# Создание строки
print(dict_to_xml_str('item', d))

# Правильное создание XML
e = dict_to_xml('item', d)
print(tostring(e))


<Element 'stock' at 0x0000020F5B71C130>
b'<stock><name>GOOG</name><shares>100</shares><price>490.1</price></stock>'
b'<stock _id="1234"><name>GOOG</name><shares>100</shares><price>490.1</price></stock>'
<item><name><spam></name></item>
b'<item><name>&lt;spam&gt;</name></item>'


In [88]:
from xml.etree.ElementTree import parse, Element

doc = parse('pred.xml')
root = doc.getroot()
print(root)

# Удалим несколько элементов
root.remove(root.find('sri')) 
root.remove(root.find('cr'))

# Вставка нового элемента после <nm>...</nm>
print(list(root).index(root.find('nm')))
e = Element('spam')
e.text = 'This is a test'
root.insert(2, e)

# Запись обратно в файл
doc.write('newpred.xml', xml_declaration=True)

<Element 'stop' at 0x0000020F5B74C090>
1


In [97]:
doc = parse('test.xml')

# Некоторые запросы, которые работают
print(doc.findtext('author'))
print(doc.find('content'))

# Запрос с использованием пространства имен (не работает)
print(doc.find('content/html'))

# Работает при полном определении
print(doc.find('content/{http://www.w3.org/1999/xhtml}html'))

# Не работает
print(doc.findtext('content/{http://www.w3.org/1999/xhtml}html/head/title'))

# Полностью определен
print(doc.findtext('content/{http://www.w3.org/1999/xhtml}html/{http://www.w3.org/1999/xhtml}head/{http://www.w3.org/1999/xhtml}title'))

David Beazley
<Element 'content' at 0x0000020F5B786200>
None
<Element '{http://www.w3.org/1999/xhtml}html' at 0x0000020F5B7862F0>
None
Hello World


In [99]:
class XMLNamespaces:
    def __init__(self, **kwargs):
        self.namespaces = {}
        for name, uri in kwargs.items():
            self.register(name, uri)
    
    def register(self, name, uri):
        self.namespaces[name] = '{'+uri+'}'
    
    def __call__(self, path):
        return path.format_map(self.namespaces)
    
ns = XMLNamespaces(html='http://www.w3.org/1999/xhtml')
print(doc.find(ns('content/{html}html')))
print(doc.findtext(ns('content/{html}html/{html}head/{html}title')))

<Element '{http://www.w3.org/1999/xhtml}html' at 0x0000020F5B7862F0>
Hello World


In [100]:
from xml.etree.ElementTree import iterparse

for evt, elem in iterparse('test.xml', ('end', 'start-ns', 'end-ns')):
    print(evt, elem)

end <Element 'author' at 0x0000020F5B7867F0>
start-ns ('', 'http://www.w3.org/1999/xhtml')
end <Element '{http://www.w3.org/1999/xhtml}title' at 0x0000020F5B7877E0>
end <Element '{http://www.w3.org/1999/xhtml}head' at 0x0000020F5B786930>
end <Element '{http://www.w3.org/1999/xhtml}h1' at 0x0000020F5B787A10>
end <Element '{http://www.w3.org/1999/xhtml}body' at 0x0000020F5B787970>
end <Element '{http://www.w3.org/1999/xhtml}html' at 0x0000020F5B7871F0>
end-ns None
end <Element 'content' at 0x0000020F5B7872E0>
end <Element 'top' at 0x0000020F5B34BF10>


In [101]:
import sqlite3

stocks = [
    ('GOOG', 100, 490.1),
    ('AAPL', 50, 545.75),
    ('FB', 150, 7.45),
    ('HPQ', 75, 33.2),
]

db = sqlite3.connect('database.db')
c = db.cursor()
c.execute('create table portfolio (symbol text, shares integer, price real)')
db.commit()

In [102]:
c.executemany('insert into portfolio values (?,?,?)', stocks)
db.commit()

In [103]:
for row in db.execute('select * from portfolio'):
    print(row)

('GOOG', 100, 490.1)
('AAPL', 50, 545.75)
('FB', 150, 7.45)
('HPQ', 75, 33.2)


In [104]:
min_price = 100
for row in db.execute('select * from portfolio where price >= ?', (min_price,)):
    print(row)

('GOOG', 100, 490.1)
('AAPL', 50, 545.75)


In [111]:
import binascii
import base64

# Изначальная байтовая строка
s = b'hello'

# Закодировать в hex
h = binascii.b2a_hex(s)
print(h)

# Декодировать обратно в байты
print(binascii.a2b_hex(h))

# base64
h = base64.b16encode(s)
print(h)
print(base64.b16decode(h))
print(h.decode('ascii'))

# Закодировать в Base64
a = base64.b64encode(s)
print(a)

# Декодировать из Base64
print(base64.b64decode(a))

b'68656c6c6f'
b'hello'
b'68656C6C6F'
b'hello'
68656C6C6F
b'aGVsbG8='
b'hello'


In [1]:
from struct import Struct

def write_records(records, format, f):
    '''Записывает последовательность кортежей в бинарный файл структур.'''
    record_struct = Struct(format)
    for r in records:
        f.write(record_struct.pack(*r))
        
records = [(1, 2.3, 4.5),
           (6, 7.8, 9.0),
           (12, 13.4, 56.7)]

with open('data.b', 'wb') as f:
    write_records(records, '<idd', f)

In [2]:
def read_records(format, f):
    record_struct = Struct(format)
    chunks = iter(lambda: f.read(record_struct.size), b'')
    return (record_struct.unpack(chunk) for chunk in chunks)

with open('data.b', 'rb') as f:
    for rec in read_records('<idd', f):
        print(rec)

(1, 2.3, 4.5)
(6, 7.8, 9.0)
(12, 13.4, 56.7)


In [3]:
def unpack_records(format, data):
    record_struct = Struct(format)
    return (record_struct.unpack_from(data, offset) for offset in range(0, len(data), record_struct.size))

with open('data.b', 'rb') as f:
    data = f.read()
    
for rec in unpack_records('<idd', data):
    print(rec)

(1, 2.3, 4.5)
(6, 7.8, 9.0)
(12, 13.4, 56.7)


In [7]:
record_struct = Struct('<idd')
print(record_struct.size)
print(record_struct.pack(1, 2.0, 3.0))
print(record_struct.unpack(record_struct.pack(1, 2.0, 3.0)))

20
b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@'
(1, 2.0, 3.0)


In [10]:
with open('data.b', 'rb') as f:
    chunks = iter(lambda: f.read(20), b'')
    print(chunks)

    for chk in chunks:
        print(chk)

b'\x01\x00\x00\x00ffffff\x02@\x00\x00\x00\x00\x00\x00\x12@'
b'\x06\x00\x00\x00333333\x1f@\x00\x00\x00\x00\x00\x00"@'
b'\x0c\x00\x00\x00\xcd\xcc\xcc\xcc\xcc\xcc*@\x9a\x99\x99\x99\x99YL@'


In [14]:
from collections import namedtuple

def read_records(format, f):
    record_struct = Struct(format)
    while True:
        chk = f.read(record_struct.size)
        if chk == b'':
            break
        yield record_struct.unpack(chk)
    return records

def unpack_records(format, data):
    record_struct = Struct(format)
    return (record_struct.unpack(data[offset:offset + record_struct.size]) for offset in range(0, len(data), record_struct.size))

Record = namedtuple('Record', ['kind','x','y'])

with open('data.b', 'rb') as f:
    records = (Record(*r) for r in read_records('<idd', f))
    
    for r in records:
        print(r.kind, r.x, r.y)


1 2.3 4.5
6 7.8 9.0
12 13.4 56.7


In [15]:
import numpy as np

with open('data.b', 'rb') as f:
    records = np.fromfile(f, dtype='<i,<d,<d')
    print(records)

[( 1,  2.3,  4.5) ( 6,  7.8,  9. ) (12, 13.4, 56.7)]


In [17]:
import struct
import itertools


polys = [
    [ (1.0, 2.5), (3.5, 4.0), (2.5, 1.5) ],
    [ (7.0, 1.2), (5.1, 3.0), (0.5, 7.5), (0.8, 9.0) ],
    [ (3.4, 6.3), (1.2, 0.5), (4.6, 9.2) ],
]

def write_polys(filename, polys):
    # Определяем ограничивающий параллелепипед
    flattened = list(itertools.chain(*polys))
    min_x = min(x for x, y in flattened)
    max_x = max(x for x, y in flattened)
    min_y = min(y for x, y in flattened)
    max_y = max(y for x, y in flattened)
    
    with open(filename, 'wb') as f:
        f.write(struct.pack('<iddddi', 0x1234, min_x, min_y, max_x, max_y, len(polys)))
        
        for poly in polys:
            size = len(poly) * struct.calcsize('<dd')
            f.write(struct.pack('<i', size+4))
            for pt in poly:
                f.write(struct.pack('<dd', *pt))
            
# Вызываем с нашими данными полигонов
write_polys('polys.bin', polys)

In [18]:
def read_polys(filename):
    with open(filename, 'rb') as f:
        # Читаем заголовок
        header = f.read(40)
        file_code, min_x, min_y, max_x, max_y, num_polys = struct.unpack('<iddddi', header)
        
        polys = []
        for n in range(num_polys):
            pbytes, = struct.unpack('<i', f.read(4))
            poly = []
            for m in range(pbytes // 16):
                pt = struct.unpack('<dd', f.read(16))
                poly.append(pt)
            polys.append(poly)
    
    return polys

read_polys('polys.bin')

[[(1.0, 2.5), (3.5, 4.0), (2.5, 1.5)],
 [(7.0, 1.2), (5.1, 3.0), (0.5, 7.5), (0.8, 9.0)],
 [(3.4, 6.3), (1.2, 0.5), (4.6, 9.2)]]

In [20]:
class StructField:
    '''Дескриптор, представляющий простое поле структуры'''
    def __init__(self, format, offset):
        self.format = format
        self.offset = offset
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            r = struct.unpack_from(self.format, instance._buffer, self.offset)
        return r[0] if len(r) == 1 else r
    

class Structure:
    def __init__(self, bytedata):
        self._buffer = memoryview(bytedata)
        
class PolyHeader(Structure):
    file_code = StructField('<i', 0)
    min_x = StructField('<d', 4)
    min_y = StructField('<d', 12)
    max_x = StructField('<d', 20)
    max_y = StructField('<d', 28)
    num_polys = StructField('<i', 36)
    
with open('polys.bin', 'rb') as f:
    phead = PolyHeader(f.read(40))
    print(phead.file_code == 0x1234)
    print(phead.min_x)
    print(phead.min_y)
    print(phead.max_x)
    print(phead.max_y)
    print(phead.num_polys)

True
0.5
0.5
7.0
9.2
3


In [21]:
class StructureMeta(type):
    '''Метакласс, который автоматически создает дескрипторы StructField'''
    def __init__(self, clsname, bases, clsdict):
        fields = getattr(self, '_fields_', [])
        byte_order = ''
        offset = 0
        
        for format, fieldname in fields:
            if format.startswith(('<','>','!','@')):
                byte_order = format[0]
                format = format[1:]
            format = byte_order + format
            setattr(self, fieldname, StructField(format, offset))
            offset += struct.calcsize(format)
        setattr(self, 'struct_size', offset)
        

class Structure(metaclass=StructureMeta):
    def __init__(self, bytedata):
        self._buffer = bytedata
        
    @classmethod
    def from_file(cls, f):
        return cls(f.read(cls.struct_size))
    

class PolyHeader(Structure):
    _fields_ = [
        ('<i', 'file_code'),
        ('d', 'min_x'),
        ('d', 'min_y'),
        ('d', 'max_x'),
        ('d', 'max_y'),
        ('i', 'num_polys')
    ]
    

with open('polys.bin', 'rb') as f:
    phead = PolyHeader(f.read(40))
    print(phead.file_code == 0x1234)
    print(phead.min_x)
    print(phead.min_y)
    print(phead.max_x)
    print(phead.max_y)
    print(phead.num_polys)    
    
    
    

True
0.5
0.5
7.0
9.2
3


In [30]:
class SizedRecord:
    def __init__(self, bytedata):
        self._buffer = memoryview(bytedata)
        
    @classmethod
    def from_file(cls, f, size_fmt, includes_size=True):
        sz_nbytes = struct.calcsize(size_fmt)
        sz_bytes = f.read(sz_nbytes)
        sz, = struct.unpack(size_fmt, sz_bytes)
        buf = f.read(sz - includes_size * sz_nbytes)
        return cls(buf)
    
    def iter_as(self, code):
        if isinstance(code, str):
            s = struct.Struct(code)
            for off in range(0, len(self._buffer), s.size):
                yield s.unpack_from(self._buffer, off)
        elif isinstance(code, StructureMeta):
            size = code.struct_size
            for off in range(0, len(self._buffer), size):
                data = self._buffer[off:off+size]
                yield code(data)
                

with open('polys.bin', 'rb') as f:
    phead = PolyHeader.from_file(f)
    print(phead.num_polys)
    
    polydata = [SizedRecord.from_file(f, '<i') for n in range(phead.num_polys)]
    print(polydata)
    
    for n, poly in enumerate(polydata):
        print('Polygon', n)
        for p in poly.iter_as('<dd'):
            print(p)

3
[<__main__.SizedRecord object at 0x000001B3FF518700>, <__main__.SizedRecord object at 0x000001B3FF620D00>, <__main__.SizedRecord object at 0x000001B3FF620AF0>]
Polygon 0
(1.0, 2.5)
(3.5, 4.0)
(2.5, 1.5)
Polygon 1
(7.0, 1.2)
(5.1, 3.0)
(0.5, 7.5)
(0.8, 9.0)
Polygon 2
(3.4, 6.3)
(1.2, 0.5)
(4.6, 9.2)
